In [13]:
%matplotlib inline

from mesa import Mesa
import argparse
from utils import Rater, load_dataset
from sklearn.tree import *
from copy import deepcopy
import time

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

parser = argparse.ArgumentParser(description='Mesa Arguments')
parser.add_argument('--env-name', default="MESA-SAC")

# SAC arguments
parser.add_argument('--policy', default="Gaussian",
                    help='Policy Type: Gaussian | Deterministic (default: Gaussian)')
parser.add_argument('--eval', type=bool, default=True,
                    help='Evaluates a policy every 10 episode (default: True)')
parser.add_argument('--gamma', type=float, default=0.99, metavar='G',
                    help='discount factor for reward (default: 0.99)')
parser.add_argument('--tau', type=float, default=0.01, metavar='G',
                    help='target smoothing coefficient(τ) (default: 0.01)')
parser.add_argument('--lr', type=float, default=0.001, metavar='G',
                    help='learning rate (default: 0.001)')
parser.add_argument('--lr_decay_steps', type=int, default=10, metavar='N',
                    help='step_size of StepLR learning rate decay scheduler (default: 10)')
parser.add_argument('--lr_decay_gamma', type=float, default=0.99, metavar='N',
                    help='gamma of StepLR learning rate decay scheduler (default: 0.99)')
parser.add_argument('--alpha', type=float, default=0.1, metavar='G',
                    help='Temperature parameter α determines the relative importance of the entropy\
                            term against the reward (default: 0.1)')
parser.add_argument('--automatic_entropy_tuning', type=bool, default=False, metavar='G',
                    help='Automaically adjust α (default: False)')
parser.add_argument('--seed', type=int, default=None, metavar='N',
                    help='random seed (default: None)')
parser.add_argument('--batch_size', type=int, default=64, metavar='N',
                    help='batch size (default: 64)')
parser.add_argument('--hidden_size', type=int, default=50, metavar='N',
                    help='hidden size (default: 50)')
parser.add_argument('--updates_per_step', type=int, default=1, metavar='N',
                    help='model updates per simul|ator step (default: 1)')
parser.add_argument('--update_steps', type=int, default=1000, metavar='N',
                    help='maximum number of steps (default: 1000)')
parser.add_argument('--start_steps', type=int, default=500, metavar='N',
                    help='Steps sampling random actions (default: 500)')
parser.add_argument('--target_update_interval', type=int, default=1, metavar='N',
                    help='Value target update per no. of updates per step (default: 1)')
parser.add_argument('--replay_size', type=int, default=1000, metavar='N',
                    help='size of replay buffer (default: 1000)')
parser.add_argument('--cuda', action="store_true", default=False,
                    help='run on CUDA (default: False)')

# MESA arguments
parser.add_argument('--dataset', type=str, default='Mammo', metavar='N',
                    help='the dataset used for meta-training (default: Mammo)')
parser.add_argument('--metric', type=str, default='aucprc', metavar='N',
                    help='the metric used for evaluate (default: aucprc)')
parser.add_argument('--reward_coefficient', type=float, default=100, metavar='N')
parser.add_argument('--num_bins', type=int, default=5, metavar='N', 
                    help='number of bins (default: 5). state-size = 2 * num_bins.')
parser.add_argument('--sigma', type=float, default=0.2, metavar='N', 
                    help='sigma of the Gaussian function used in meta-sampling (default: 0.2)')
parser.add_argument('--max_estimators', type=int, default=10, metavar='N',
                    help='maximum number of base estimators in each meta-training episode (default: 10)')
parser.add_argument('--meta_verbose', type=int, default=10, metavar='N',
                    help='number of episodes between verbose outputs. \
                    If \'full\' print log for each base estimator (default: 10)')
parser.add_argument('--meta_verbose_mean_episodes', type=int, default=25, metavar='N',
                    help='number of episodes used for compute latest mean score in verbose outputs.')
parser.add_argument('--verbose', type=bool, default=False, metavar='N',
                    help='enable verbose when ensemble fit (default: False)')
parser.add_argument('--random_state', type=int, default=None, metavar='N', 
                    help='random_state (default: None)')
parser.add_argument('--train_ir', type=float, default=1, metavar='N', 
                    help='imbalance ratio of the training set after meta-sampling (default: 1)')
parser.add_argument('--train_ratio', type=float, default=1, metavar='N', 
                    help='the ratio of the data used in meta-training. \
                    set train_ratio<1 to use a random subset for meta-training (default: 1)')

_StoreAction(option_strings=['--train_ratio'], dest='train_ratio', nargs=None, const=None, default=1, type=<class 'float'>, choices=None, required=False, help='the ratio of the data used in meta-training.                     set train_ratio<1 to use a random subset for meta-training (default: 1)', metavar='N')

# Initialization and meta-training

In [ ]:
''' Prepare the Environment '''
#from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold

#  load dataset\
data = pd.read_csv("data/df_train_lowdi.csv")
X = data.drop(columns=["飆股"])  
y = data["飆股"]
'''
# 分割資料集
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

#estimator, base_estimator = 'XGB', XGBClassifier(use_label_encoder=False, eval_metric='logloss')
estimator, base_estimator = 'lgb', LGBMClassifier(objective='binary', metric='logloss', force_col_wise=True, verbose = -1)

args = parser.parse_args([])
n_estimators = args.max_estimators
'''
# Initialize model and parameters
args = parser.parse_args([])
#estimator, base_estimator = 'XGB', XGBClassifier(use_label_encoder=False, eval_metric='logloss')
estimator, base_estimator = 'lgb', LGBMClassifier(objective='binary', metric='logloss', force_col_wise=True, verbose = -1)
n_estimators = args.max_estimators

n_splits = 3
n_trials = 10  # Optuna 的試驗次數，可以根據需要調整
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)


In [15]:
data = data.dropna(subset=['飆股'])
X = data.drop(columns=['飆股'])
y = data['飆股']
print(y.isna().sum())  # 檢查 y 中有多少 NaN

0


### Optuna

In [ ]:
# objective
from sklearn.metrics import average_precision_score 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
def objective(trial, X, y, n_splits=5):
    # 定義超參數搜索空間
    params = {
        "objective": "binary",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "bagging_freq": 1,
        "metric": "",  # 清空原生metric，使用自定義評估函數
        "num_threads": 4,
        
        # 模型結構參數
        "max_depth": trial.suggest_int("max_depth", 3, 11),
        "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        
        # 學習與正則化參數
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-9, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-9, 10.0, log=True),
        
        # 採樣參數
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
    }

    # 初始化交叉驗證
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    cv_scores = []

    # 交叉驗證迴圈
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
        # 分割數據
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        X_train_t, X_valid_t, y_train_t, y_valid_t = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

        # 初始化 LGBMClassifier
        base_estimator = LGBMClassifier(**params)

        # 初始化 Mesa
        mesa = Mesa(
            args=args,
            base_estimator=base_estimator,
            n_estimators=args.max_estimators
        )

        # 訓練 Mesa
        mesa.meta_fit(X_train_t, y_train_t, X_valid_t, y_valid_t, None, None)

        # 預測驗證集
        y_pred = mesa.predict_proba(X_valid)[:, 1]  # 假設 Mesa 有 predict_proba 方法

        # 計算 AUPRC
        auprc = average_precision_score(y_valid, y_pred)
        cv_scores.append(auprc)

    # 返回交叉驗證的平均 AUPRC
    return np.mean(cv_scores)

In [5]:
import optuna
study = optuna.create_study(direction='maximize')  # 最大化 AUPRC
study.optimize(lambda trial: objective(trial, X, y, n_splits), n_trials=n_trials)

# 輸出最佳結果
print(f"Best trial: {study.best_trial.number}")
print(f"Best AUPRC: {study.best_value:.4f}")
print(f"Best parameters: {study.best_params}")

# 使用最佳參數重新訓練最終模型
best_params = study.best_params
base_estimator = LGBMClassifier(
    objective='binary',
    metric='logloss',
    force_col_wise=True,
    verbose=-1,
    **best_params
)

# 初始化最終的 Mesa 模型
mesa_final = Mesa(
    args=args,
    base_estimator=base_estimator,
    n_estimators=args.max_estimators)


# 在完整訓練數據上訓練
start_time = time.time()
print("Training final Mesa model with best parameters...")
mesa_final.meta_fit(X, y, X, y, None, None) # 假設使用全數據訓練
end_time = time.perf_counter()
print(f"Final training time: {end_time - start_time:.3f} s")

# 保存結果
df_results = pd.DataFrame({
    'best_auprc': [study.best_value],
    'best_params': [study.best_params]
})
df_results.to_csv('results/optuna_results.csv', index=False)

[I 2025-04-15 01:40:34,506] A new study created in memory with name: no-name-fdd60f76-73d2-48ba-b9e5-5e5a0165fade


Epi.10   updates 0    |last-25-mean-aucprc| train 0.348 | valid 0.280 | by rand
Epi.20   updates 0    |last-25-mean-aucprc| train 0.343 | valid 0.275 | by rand
Epi.30   updates 0    |last-25-mean-aucprc| train 0.336 | valid 0.269 | by rand
Epi.40   updates 0    |last-25-mean-aucprc| train 0.340 | valid 0.272 | by rand
Epi.50   updates 0    |last-25-mean-aucprc| train 0.357 | valid 0.284 | by rand
Epi.60   updates 40   |last-25-mean-aucprc| train 0.365 | valid 0.288 | by mesa
Epi.70   updates 130  |last-25-mean-aucprc| train 0.385 | valid 0.302 | by mesa
Epi.80   updates 220  |last-25-mean-aucprc| train 0.425 | valid 0.328 | by mesa
Epi.90   updates 310  |last-25-mean-aucprc| train 0.464 | valid 0.353 | by mesa
Epi.100  updates 400  |last-25-mean-aucprc| train 0.478 | valid 0.360 | by mesa
Epi.110  updates 490  |last-25-mean-aucprc| train 0.483 | valid 0.365 | by mesa
Epi.120  updates 580  |last-25-mean-aucprc| train 0.485 | valid 0.365 | by mesa
Epi.130  updates 670  |last-25-mean-aucp

[I 2025-04-15 02:15:23,725] Trial 0 finished with value: 0.37545595741301696 and parameters: {'max_depth': 6, 'num_leaves': 947, 'min_data_in_leaf': 50, 'learning_rate': 0.0073452724505615645, 'lambda_l1': 7.026912294986481e-08, 'lambda_l2': 1.2659307296925426, 'subsample': 0.6133308686022046, 'colsample_bytree': 0.3901999606909041}. Best is trial 0 with value: 0.37545595741301696.


Epi.10   updates 0    |last-25-mean-aucprc| train 0.232 | valid 0.187 | by rand
Epi.20   updates 0    |last-25-mean-aucprc| train 0.236 | valid 0.192 | by rand
Epi.30   updates 0    |last-25-mean-aucprc| train 0.236 | valid 0.193 | by rand
Epi.40   updates 0    |last-25-mean-aucprc| train 0.232 | valid 0.191 | by rand
Epi.50   updates 0    |last-25-mean-aucprc| train 0.226 | valid 0.187 | by rand
Epi.60   updates 40   |last-25-mean-aucprc| train 0.229 | valid 0.188 | by mesa
Epi.70   updates 130  |last-25-mean-aucprc| train 0.248 | valid 0.200 | by mesa
Epi.80   updates 220  |last-25-mean-aucprc| train 0.276 | valid 0.217 | by mesa
Epi.90   updates 310  |last-25-mean-aucprc| train 0.298 | valid 0.232 | by mesa
Epi.100  updates 400  |last-25-mean-aucprc| train 0.302 | valid 0.234 | by mesa
Epi.110  updates 490  |last-25-mean-aucprc| train 0.306 | valid 0.235 | by mesa
Epi.120  updates 580  |last-25-mean-aucprc| train 0.308 | valid 0.236 | by mesa
Epi.130  updates 670  |last-25-mean-aucp

[I 2025-04-15 02:42:54,713] Trial 1 finished with value: 0.24915585076566252 and parameters: {'max_depth': 4, 'num_leaves': 106, 'min_data_in_leaf': 9, 'learning_rate': 0.004794663135372045, 'lambda_l1': 2.4476553165341892e-05, 'lambda_l2': 5.1501550973065044e-05, 'subsample': 0.09388852716834512, 'colsample_bytree': 0.09249437025589183}. Best is trial 0 with value: 0.37545595741301696.


...

[I 2025-04-15 06:56:36,730] Trial 8 finished with value: 0.43856211786042604 and parameters: {'max_depth': 9, 'num_leaves': 998, 'min_data_in_leaf': 40, 'learning_rate': 0.06250085707237722, 'lambda_l1': 4.329573736564691, 'lambda_l2': 0.3316413224175226, 'subsample': 0.49401108761863277, 'colsample_bytree': 0.9486630908063604}. Best is trial 2 with value: 0.614977695736123.


Epi.10   updates 0    |last-25-mean-aucprc| train 0.227 | valid 0.205 | by rand
Epi.20   updates 0    |last-25-mean-aucprc| train 0.225 | valid 0.203 | by rand
Epi.30   updates 0    |last-25-mean-aucprc| train 0.224 | valid 0.202 | by rand
Epi.40   updates 0    |last-25-mean-aucprc| train 0.224 | valid 0.201 | by rand
Epi.50   updates 0    |last-25-mean-aucprc| train 0.224 | valid 0.202 | by rand
Epi.60   updates 40   |last-25-mean-aucprc| train 0.222 | valid 0.201 | by mesa
Epi.70   updates 130  |last-25-mean-aucprc| train 0.231 | valid 0.204 | by mesa
Epi.80   updates 220  |last-25-mean-aucprc| train 0.242 | valid 0.209 | by mesa
Early stopping triggered after 80 episodes. Best validation score: 0.223
Epi.10   updates 0    |last-25-mean-aucprc| train 0.213 | valid 0.186 | by rand
Epi.20   updates 0    |last-25-mean-aucprc| train 0.209 | valid 0.182 | by rand
Epi.30   updates 0    |last-25-mean-aucprc| train 0.208 | valid 0.179 | by rand
Epi.40   updates 0    |last-25-mean-aucprc| tra

[I 2025-04-15 07:11:29,797] Trial 9 finished with value: 0.21490274064805048 and parameters: {'max_depth': 10, 'num_leaves': 310, 'min_data_in_leaf': 89, 'learning_rate': 0.0026367682271810377, 'lambda_l1': 2.0077261185869212e-05, 'lambda_l2': 1.2153584286317087, 'subsample': 0.34721613985500044, 'colsample_bytree': 0.9026314046590259}. Best is trial 2 with value: 0.614977695736123.


Best trial: 2
Best AUPRC: 0.6150
Best parameters: {'max_depth': 6, 'num_leaves': 671, 'min_data_in_leaf': 10, 'learning_rate': 0.018895029642954978, 'lambda_l1': 0.5808032338650604, 'lambda_l2': 2.8808120188637366e-05, 'subsample': 0.756846781197452, 'colsample_bytree': 0.40525672612507596}
Training final Mesa model with best parameters...
Epi.10   updates 0    |last-25-mean-aucprc| train 0.610 | valid 0.610 | by rand
Epi.20   updates 0    |last-25-mean-aucprc| train 0.640 | valid 0.640 | by rand
Epi.30   updates 0    |last-25-mean-aucprc| train 0.629 | valid 0.629 | by rand
Epi.40   updates 0    |last-25-mean-aucprc| train 0.632 | valid 0.632 | by rand
Epi.50   updates 0    |last-25-mean-aucprc| train 0.611 | valid 0.611 | by rand
Epi.60   updates 40   |last-25-mean-aucprc| train 0.636 | valid 0.636 | by mesa
Epi.70   updates 130  |last-25-mean-aucprc| train 0.724 | valid 0.724 | by mesa
Epi.80   updates 220  |last-25-mean-aucprc| train 0.815 | valid 0.815 | by mesa
Epi.90   updates 3

OSError: Cannot save file into a non-existent directory: 'results'

### 沒CV

In [19]:
from sklearn.model_selection import train_test_split

# 初始化最終的 Mesa 模型
mesa_final = Mesa(
    args=args,
    base_estimator=base_estimator,
    n_estimators=args.max_estimators)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
# 在完整訓練數據上訓練
start_time = time.time()
print("Training final Mesa model with best parameters...")
mesa_final.meta_fit(X_train, y_train, X_valid, y_valid, None, None) # 假設使用全數據訓練
end_time = time.perf_counter()
print(f"Final training time: {end_time - start_time:.3f} s")

Training final Mesa model with best parameters...
Epi.10   updates 0    |last-25-mean-aucprc| train 0.980 | valid 0.734 | by rand
Epi.20   updates 0    |last-25-mean-aucprc| train 0.968 | valid 0.716 | by rand
Epi.30   updates 0    |last-25-mean-aucprc| train 0.967 | valid 0.710 | by rand
Epi.40   updates 0    |last-25-mean-aucprc| train 0.973 | valid 0.713 | by rand
Epi.50   updates 0    |last-25-mean-aucprc| train 0.987 | valid 0.736 | by rand
Epi.60   updates 40   |last-25-mean-aucprc| train 0.989 | valid 0.747 | by mesa
Epi.70   updates 130  |last-25-mean-aucprc| train 0.993 | valid 0.776 | by mesa
Epi.80   updates 220  |last-25-mean-aucprc| train 0.998 | valid 0.822 | by mesa
Epi.90   updates 310  |last-25-mean-aucprc| train 1.000 | valid 0.846 | by mesa
Epi.100  updates 400  |last-25-mean-aucprc| train 1.000 | valid 0.849 | by mesa
Epi.110  updates 490  |last-25-mean-aucprc| train 1.000 | valid 0.850 | by mesa
Epi.120  updates 580  |last-25-mean-aucprc| train 1.000 | valid 0.850 

KeyboardInterrupt: 

In [8]:
publicdata = pd.read_csv("data/df_public_lowdi.csv")
privatedata = pd.read_csv("data/df_private_lowdi.csv")

In [ ]:
predictions = mesa_final.predict(publicdata)
results1 = pd.DataFrame({
    #'ID': sub['ID'], 
    '飆股': predictions
})
results1.to_csv('result/predictions_public.csv', index=False)

probas = mesa_final.predict_proba(publicdata)
#是飆股的機率
positive_probas = probas[:, 1]
results_prob1 = pd.DataFrame({
    #'ID': pub['ID'], 
    '飆股機率': positive_probas  # 正類的機率
})
results_prob1.to_csv('result/predictions_prob_public.csv', index=False)

In [ ]:
predictions = mesa_final.predict(privatedata)
results2 = pd.DataFrame({
    #'ID': pri['ID'], 
    '飆股': predictions
})
results2.to_csv('result/predictions_private.csv', index=False)

probas = mesa_final.predict_proba(privatedata)
#是飆股的機率
positive_probas = probas[:, 1]
results_prob2 = pd.DataFrame({
    #'ID': pri['ID'], 
    '飆股機率': positive_probas  # 正類的機率
})
results_prob2.to_csv('result/predictions_prob_private.csv', index=False)

In [20]:
sub = pd.read_csv("data/submission_template_public_and_private.csv")

In [21]:
result = pd.concat([results1, results2], axis=0, ignore_index=True)
sub['飆股'] = result['飆股']
sub.to_csv('result/predictions_lowdi_xgb.csv', index=False)

In [17]:
preddata = pd.read_csv("result/predictions_optuna.csv")
preddata.describe()

,飆股
count,25108.000000
mean,0.012705
std,0.112001
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000
